### Hyperparameter Tuning.

* Hyperparameter tuning in machine learning is the process of optimizing  the hperparameters of a learning algorithm to improve its performance.
  
#### Tuning methods
- Grid Search
- Random Search

#### examples

- Same model with differnt hyperparameters.
- Different models with different hyperparameters.


In [1]:
from sklearn import svm, datasets
iris = datasets.load_iris()

In [3]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV( svm.SVC(gamma='auto'), {
    'C': [1,20,20],
    'kernel': ['rbf', 'linear']
}, cv=5, return_train_score=False)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00198998, 0.00057554, 0.0006186 , 0.00056477, 0.00063844,
        0.00049763]),
 'std_fit_time': array([1.50013946e-03, 1.24335415e-04, 4.85228611e-05, 7.03168138e-05,
        1.38799950e-04, 9.97991808e-06]),
 'mean_score_time': array([0.00126791, 0.00040593, 0.00041962, 0.00041685, 0.00041537,
        0.00032673]),
 'std_score_time': array([9.60206099e-04, 7.31709066e-05, 1.63465453e-05, 1.22216778e-04,
        3.35570741e-05, 7.27797853e-06]),
 'param_C': masked_array(data=[1, 1, 20, 20, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'},
  {'C': 20, 'ker

In [4]:
import pandas as pd
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001990,0.001500,0.001268,0.000960,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000576,0.000124,0.000406,0.000073,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000619,0.000049,0.000420,0.000016,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,3
3,0.000565,0.000070,0.000417,0.000122,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,5
4,0.000638,0.000139,0.000415,0.000034,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,3
5,0.000498,0.000010,0.000327,0.000007,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,5


In [5]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,20,rbf,0.966667
3,20,linear,0.966667
4,20,rbf,0.966667
5,20,linear,0.966667


In [6]:
clf.best_score_

0.9800000000000001

In [8]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

### RandomizedSearchCV

In [11]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,20,linear,0.966667


### How about different models with different hyperparameters?

In [12]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [13]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}
